In [2]:
import functools as ft
import io
import itertools
import json
from pathlib import Path

import numpy as np
import matplotlib
from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sns

from matplotlib.axes import Axes
from tqdm.notebook import tqdm

import matplotlib.font_manager as fm

In [3]:
sns.set_theme(
    "paper",
    "darkgrid",
    font_scale=1.1,
    rc={
        "font.family": "serif",
        "font.serif": ["CMU Concrete"],
        "axes.facecolor": ".95",
        # math font
        "mathtext.fontset": "cm",
        # tex setting
        "text.usetex": False,
    },
)

In [ ]:
dataset_name = "karate"
# load results/{dataset_name}/{algo_name}/{id}.txt
# each file contains 10 lines, each line has two numbers
# corresponding to a vertex and a sample complexity
# also results/{dataset_name}/{algo_name}/{id}_eval.txt
# each line has one number: mean of evaluation
#
# generate a dataframe with columns
# - algo_name
# - id
# - vertex
# - sample_complexity
# - evaluation

df = pd.DataFrame()

for algo_name in ["greedy", "celf", "greedy-cb", "celf-cb"]:
  for id in range(1, 1024 + 1):
    df1 = pd.read_csv(f"results/{dataset_name}/{algo_name}/{id}.txt", header=None)
    df1.columns = ["vertex", "sample_complexity"]
    # eval_df1 = pd.read_csv(f"results/{dataset_name}/{algo_name}/{id}_eval.txt", header=None)
    # eval_df1.columns = ["evaluation"]
    # df1 = pd.concat([df1, eval_df1], axis=1)
    df1["algo_name"] = algo_name
    df1["id"] = id
    df1["dataset_name"] = dataset_name

    df = pd.concat([df, df1], axis=0)